In [1]:
# Import Packages
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
from keras.optimizers import Adam
from models import model
from utils import train_val_generator
from matplotlib import pyplot as plt
from keras.models import load_model
from os.path import isfile

Using TensorFlow backend.


In [2]:
# Training Parameters
choices=['vgg_16','vgg_19','resnet_152','simple'] # All Models Available
model_name = choices[1]

is_transfer = True
num_freeze_layer = 6
num_classes = 4
weights_path = 'resnet152_weights_tf.h5'
input_shape = (224,224,3) # Input Shape for Resnet152

In [3]:
# Path to Data
train_path = './train'
test_path ='./test'

In [4]:
# Model for Traning
tr_model = model(model_name,num_classes,is_transfer,
                num_freeze_layer,weights_path,input_shape)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________

In [5]:
# Train and Test generator
train_gen, val_gen = train_val_generator(32,train_path,test_path)

Found 42994 images belonging to 4 classes.
Found 7031 images belonging to 4 classes.


In [6]:
# Load last model if exists
#model_name = model_name+'.h5'
if isfile(model_name+".h5"):
    print('Loading previously trained weights and continue traning.....')
    tr_model = load_model(model_name+".h5")
else:
    print('No saved weights found.')
    
print(model_name+".h5")

Loading previously trained weights and continue traning.....

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
vgg_19.h5


In [7]:
# Model saving
checkpoint = ModelCheckpoint(model_name+'.h5',monitor='val_acc',verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor='val_acc',min_delta=0,patience=5,verbose=1,mode='auto')

# Compile the model
tr_model.compile(loss='categorical_crossentropy',optimizer=Adam(1e-5),metrics=['accuracy'])


In [ ]:
# Train the model
history = tr_model.fit_generator(
                train_gen,
                steps_per_epoch=1400,
                epochs=30,
                validation_data = val_gen,
                validation_steps = 250,
                callbacks = [checkpoint,early_stop])

Epoch 1/30
1400/1400 [==============================] - 863s 617ms/step - loss: 0.1563 - acc: 0.9418 - val_loss: 0.1565 - val_acc: 0.9464

Epoch 00001: val_acc improved from -inf to 0.94644, saving model to vgg_19.h5
Epoch 2/30
 535/1400 [==========>...................] - ETA: 8:10 - loss: 0.1429 - acc: 0.9482

In [ ]:
tr_model.save("vanilla.h5")